In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load the training data
train_path = "/home/data/train.json"

# Read the JSON file
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of training samples: {len(train_data)}")
print(f"Type of data: {type(train_data)}")
print(f"First sample keys: {list(train_data[0].keys())}")

Number of training samples: 2878
Type of data: <class 'list'>
First sample keys: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subredd

In [2]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(train_data)

# Basic info about the dataset
print("Dataset shape:", df.shape)
print("\nColumn types:")
print(df.dtypes.value_counts())

# Check target distribution
print("\nTarget distribution:")
target_counts = df['requester_received_pizza'].value_counts()
print(target_counts)
print(f"Success rate: {target_counts[True] / len(df):.2%}")

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum().sum())

Dataset shape: (2878, 32)

Column types:
int64      16
object      9
float64     6
bool        1
Name: count, dtype: int64

Target distribution:
requester_received_pizza
False    2163
True      715
Name: count, dtype: int64
Success rate: 24.84%

Missing values:
2163


In [ ]:
# Explore text data characteristics
print("Text data exploration:")
print(f"Average request_text length: {df['request_text'].str.len().mean():.0f} characters")
print(f"Average request_title length: {df['request_title'].str.len().mean():.0f} characters")

# Sample some text
print("\nSample request title:")
print(df['request_title'].iloc[0])
print("\nSample request text (first 200 chars):")
print(df['request_text'].iloc[0][:200])

# Check edit-aware text
print("\nSample edit-aware text (first 200 chars):")
print(df['request_text_edit_aware'].iloc[0][:200])

# Check if edit_aware differs from original
print(f"\nNumber of samples where edit_aware differs from original: {(df['request_text'] != df['request_text_edit_aware']).sum()}")